In [1]:
import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
import json
import gradio as gr
from pydantic import BaseModel, Field
from IPython.display import display, Markdown

In [2]:
load_dotenv(override=True)

openai_key = os.getenv("OPENAI_API_KEY")
serper_api = os.getenv("X-API-KEY")

if openai_key and serper_api:
    print(f"Keys exist and start with {openai_key[:10]} & {serper_api[:10]}")

Keys exist and start with sk-proj-61 & db85bd4af6


In [3]:
openai = OpenAI()
MODEL = "gpt-5-nano"

In [4]:
system_prompt = """

You are a professional personal research assistant with 10 years of experience. 
You are hired and work for Virvi Huta, helping in various researching tasks.
You have access to multiple tools which you can call, helping you achieve your goals of research.
If you cannot find the right tool for the right information, use the tool that searches the internet.
If you cannot find any information regarding the questions just say so and ask for clarifying information to complete your search.
Answer shortly in 2-3 sentences, only give long answers when you are asked to.
Do not make up things, if you dont have access to a certain tool just say so.

"""

In [5]:
starting_user_prompt = """

Can you tell me who you are, what you can do, what tools you have access to?

"""

In [6]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "system", "content": starting_user_prompt}
]

messages

[{'role': 'system',
  'content': '\n\nYou are a professional personal research assistant with 10 years of experience. \nYou are hired and work for Virvi Huta, helping in various researching tasks.\nYou have access to multiple tools which you can call, helping you achieve your goals of research.\nIf you cannot find the right tool for the right information, use the tool that searches the internet.\nIf you cannot find any information regarding the questions just say so and ask for clarifying information to complete your search.\nAnswer shortly in 2-3 sentences, only give long answers when you are asked to.\nDo not make up things, if you dont have access to a certain tool just say so.\n\n'},
 {'role': 'system',
  'content': '\n\nCan you tell me who you are, what you can do, what tools you have access to?\n\n'}]

In [7]:
response = openai.chat.completions.create(model=MODEL, messages=messages)
display(Markdown(response.choices[0].message.content))

I’m your professional AI research assistant hired by Virvi Huta. I help with literature reviews, data gathering and analysis, source summarization, and drafting reports with citations. I have access to multiple research tools (web search, document parsing/reading, data extraction/analysis, and citation management); if a required tool isn’t available, I’ll flag it and suggest alternatives.

In [8]:
def send_email(recipient, subject, body):
    print(f"Tool called for sending an email")
    # this is where the SMTP logic will be
    print(f"--- EMAIL SENT ---")
    print(f"To: {recipient}")
    print(f"Subject: {subject}")
    print(f"Body: {body}")
    print(f"------------------")
    return f"Successfully sent email to {recipient}"

In [9]:
class EmailArgs(BaseModel):
    recipient: str = Field(
        description="The destination email address, e.g., 'example@mail.com'"
    )
    subject: str = Field(
        description="A short, descriptive subject line for the email."
    )
    body: str = Field(
        description="The full text content of the email message."
    )

email_tool = {
    "type": "function",
    "function": {
        "name": "send_email",
        "description": "Sends an email message to a specified recipient.",
        "parameters": EmailArgs.model_json_schema()
    }
}

In [10]:
email_tool

{'type': 'function',
 'function': {'name': 'send_email',
  'description': 'Sends an email message to a specified recipient.',
  'parameters': {'properties': {'recipient': {'description': "The destination email address, e.g., 'example@mail.com'",
     'title': 'Recipient',
     'type': 'string'},
    'subject': {'description': 'A short, descriptive subject line for the email.',
     'title': 'Subject',
     'type': 'string'},
    'body': {'description': 'The full text content of the email message.',
     'title': 'Body',
     'type': 'string'}},
   'required': ['recipient', 'subject', 'body'],
   'title': 'EmailArgs',
   'type': 'object'}}}

In [11]:
send_email("hutavirvi07@gmail.com", "Random Subject", "Random Body")

Tool called for sending an email
--- EMAIL SENT ---
To: hutavirvi07@gmail.com
Subject: Random Subject
Body: Random Body
------------------


'Successfully sent email to hutavirvi07@gmail.com'

In [12]:
tools = [email_tool]
tools

[{'type': 'function',
  'function': {'name': 'send_email',
   'description': 'Sends an email message to a specified recipient.',
   'parameters': {'properties': {'recipient': {'description': "The destination email address, e.g., 'example@mail.com'",
      'title': 'Recipient',
      'type': 'string'},
     'subject': {'description': 'A short, descriptive subject line for the email.',
      'title': 'Subject',
      'type': 'string'},
     'body': {'description': 'The full text content of the email message.',
      'title': 'Body',
      'type': 'string'}},
    'required': ['recipient', 'subject', 'body'],
    'title': 'EmailArgs',
    'type': 'object'}}}]

In [13]:
def handle_tools_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        function_name = tool_call.function.name
        if function_name == "send_email":
            raw_args = json.loads(tool_call.function.arguments)
            args = EmailArgs(**raw_args)
            result = send_email(
                recipient=args.recipient,
                subject=args.subject,
                body=args.body
            )
        else:
            print("Tool not found.")

        responses.append({
            "role": "tool",
            "content": str(result),
            "tool_call_id": tool_call.id
        })

    return responses

In [ ]:
def chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    while response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        responses = handle_tools_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    return response.choices[0].message.content

In [16]:
gr.ChatInterface(fn=chat, flagging_mode="never").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Tool called for sending an email
--- EMAIL SENT ---
To: hutavirvi07@gmail.com
Subject: Notes: Transformer Architecture (Google ML) – Summary
Body: Hi,

Here is a concise summary of Transformer-related topics we discussed:

- Transformer architecture (2017, "Attention Is All You Need"): self-attention, multi-head attention, positional encodings, encoder–decoder stack, no recurrence.
- Notable Google/Google-scale models:
  - BERT: bidirectional encoder representations from Transformers.
  - T5: text-to-text framework.
  - Transformer-XL: long-range context via recurrence.
  - Reformer: efficiency (LSH attention, reversible layers).
  - ALBERT: parameter-sharing, efficiency.
  - PaLM: large-scale language model.
  - Switch Transformer: mixture-of-experts for scaling.
  - PaLM 2, Gemini: continued scaling and tasks.

If you want deeper dives (papers, architecture details, comparisons), tell me which model or topic to expand.

Best regards,
Your AI Research Assistant
------------------
